# 1 Setting Environment

In [2]:
import pandas as pd
import numpy as np
import random

from faker import Faker
f = Faker()

import datetime
from datetime import timedelta, datetime

# 2 Transition Probabilty Matrix from CSV - SQL - CSV file

In [3]:
df = pd.read_csv('./data/weekly_loc1_data.csv', sep=",", index_col='timestamp', parse_dates=True)
df.sort_index(inplace = True)

# creating one minute intervals
df = df.groupby('cust_id').resample('1min').ffill()

# shifting to get location 2
df['location_2'] = df['location_1'].shift(-1)

#removing exit to exit state
df = df[df.location_1 != 'exit']

In [4]:
# using pandas to create probability matrix dataframe
prob_df = pd.crosstab(df['location_1'],df['location_2'], normalize='index',)
prob_df.round(2)

#converting 'matrix' to dictionary of values only
probs = prob_df.round(2).to_dict(orient='index')

for key in probs.keys():
    probs[key] = list(probs[key].values())

probs['exit'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [5]:
# declare possible states in a list 

ALL_STATES = ['checkout', 'dairy', 'drinks', 'exit', 'fruit', 'spices']

In [39]:
class Customer:
    """
    a single customer that moves through the supermarket
    in MCMC simulation
    """

    def __init__(self, customer_id):
        self.name = f.name()
        self.customer_id = customer_id
        self.state = 'entrance'
        # self.is_active = true

    # def get_name(self):
    #     return self.name

    # def get_id(self):
    #     return self.customer_id    

    def next_state(self):
        """ moves customer to next state via active status """
        if self.is_active == True:
            self.state = random.choices(population=ALL_STATES, weights=probs[state])[0]
            self.is_active()
            # customer set as active or inactive with each state transition

    # def get_state(self):
    #     return self.state    
 
    def is_active(self) -> bool: 
        if self.state != 'exit':
            return True

    def __repr__(self) -> str:
        return f'{self.name}'

In [47]:
class Supermarket:
    """
    an MCMC simulation that moves customers throughout the store during store opening times
    """

    def __init__(self, market_name, opening,  closing, max_customers=15, ave_new_customers=5):
        self.market_name = market_name
        self.opening = datetime.strptime(opening, '%H:%M')
        self.closing = datetime.strptime(closing, '%H:%M')
        self.max_customers = max_customers
        self.ave_new_customers = ave_new_customers
            #---#
        self.active_customers = []
        self.customer_id = 0
        self.current_time = self.opening
        self.customer_list = []
        self.customer_daily_count = 0
        self.customer_df = pd.DataFrame()
        
    def open_store(self):
        """ 
        initialises the store to start counter and begin accepting customers  
        """
        self.open_for_business = True
        self.timer()
        self.get_customers()

    def open_for_business(self):
        """limiting setting to stop new customers entering store too late and stop too many being in store at same time """
        while current_time != self.closing: #- timedelta(minutes=5): #or customer_limit = false
            return True

    # def customer_limit(self):
        #if len(self.active_customers) >= 15
            #return True
            
    def timer(self):
        while self.current_time != self.closing: 
            self.current_time += timedelta(minutes=1)
            self.get_customers()
            self.move_customers()
            self.record_customers()
            #self.list_customers()
        
    def get_customers(self):
        if self.open_for_business == True:
            n = np.random.poisson(self.ave_new_customers)
            for i in range(n):
                self.customer_id += 1
                self.customer_daily_count += 1
                new_customer = Customer(self.customer_id)
                self.customer_list.append(new_customer)
                self.active_customers.append(new_customer)
                
    #def list_customers(self):

    # THIS PART SHOULD BE IN CUSTOMER CLASS AND JUST CALLED UPON DURING EACH TIME STEP UP
    def move_customers(self):
        """ move customers to next state, if exit then remove from active list """
        for cust in self.active_customers:
            cust.next_state()
            cust.is_active()
            if cust.state == 'exit':
                self.active_customers.remove(cust)

    def get_time(self):
        return self.current_time

    def record_customers(self):
        for customer in self.active_customers:
            self.customer_df = self.customer_df.append({'timestamp': self.get_time(),
                                                        'customer': customer.name,
                                                        'customer_id': customer_id,
                                                        'location': customer.state
                                                        }, ignore_index=True)

# new customers generated, next step is to change customer state with each 

In [48]:
Asda = Supermarket('Asda', '07:00', '07:15')
Asda.open_store()

In [ ]:
Asda.active_customers

# 3 Creating Dataframe From Original CSV

In [74]:

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 39795 entries, 2019-09-02 07:02:00 to 2019-09-06 21:52:00
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cust_id     39795 non-null  object
 1   location_1  39795 non-null  object
dtypes: object(2)
memory usage: 932.7+ KB


In [77]:
df.loc[df['cust_id']=='01-1']

,cust_id,location_1
timestamp,,
2019-09-02 07:02:00,01-1,entrance
2019-09-02 07:03:00,01-1,dairy
2019-09-02 07:05:00,01-1,checkout
2019-09-02 07:06:00,01-1,exit


In [78]:
df = df.groupby('cust_id').resample('1min').ffill()

In [79]:
df

cust_id location_1
cust_id timestamp                             
01-1    2019-09-02 07:02:00    01-1   entrance
        2019-09-02 07:03:00    01-1      dairy
        2019-09-02 07:04:00    01-1      dairy
        2019-09-02 07:05:00    01-1   checkout
        2019-09-02 07:06:00    01-1       exit
...                             ...        ...
05-999  2019-09-06 17:24:00  05-999     drinks
        2019-09-06 17:25:00  05-999      fruit
        2019-09-06 17:26:00  05-999      fruit
        2019-09-06 17:27:00  05-999   checkout
        2019-09-06 17:28:00  05-999       exit

[68821 rows x 2 columns]

In [80]:
# shifting to get location 2
df['location_2'] = df['location_1'].shift(-1)

In [81]:
df.loc[df['cust_id']=='01-1']

cust_id location_1 location_2
cust_id timestamp                                        
01-1    2019-09-02 07:02:00    01-1   entrance      dairy
        2019-09-02 07:03:00    01-1      dairy      dairy
        2019-09-02 07:04:00    01-1      dairy   checkout
        2019-09-02 07:05:00    01-1   checkout       exit
        2019-09-02 07:06:00    01-1       exit   entrance

In [82]:
#removing exit to exit state
df = df[df.location_1 != 'exit']

In [83]:
df.head()

cust_id location_1 location_2
cust_id timestamp                                        
01-1    2019-09-02 07:02:00    01-1   entrance      dairy
        2019-09-02 07:03:00    01-1      dairy      dairy
        2019-09-02 07:04:00    01-1      dairy   checkout
        2019-09-02 07:05:00    01-1   checkout       exit
01-10   2019-09-02 07:05:00   01-10   entrance      fruit

# 4 Probability Matrix

In [84]:
# using pandas to create probability matrix dataframe
prob_df = pd.crosstab(df['location_1'],df['location_2'], normalize='index',)
prob_df.round(2)

location_2,checkout,dairy,drinks,exit,fruit,spices
location_1,,,,,,
checkout,0.00,0.00,0.00,1.0,0.00,0.00
dairy,0.10,0.74,0.06,0.0,0.05,0.05
drinks,0.22,0.01,0.60,0.0,0.09,0.09
entrance,0.00,0.29,0.15,0.0,0.38,0.18
fruit,0.20,0.10,0.05,0.0,0.60,0.05
spices,0.15,0.19,0.16,0.0,0.09,0.40


In [85]:
# checking probabilities sum to 1
sum_check = prob_df.sum(axis=1)
sum_check

location_1
checkout    1.0
dairy       1.0
drinks      1.0
entrance    1.0
fruit       1.0
spices      1.0
dtype: float64

In [86]:
#converting 'matrix' to dictionary of values only
probs = prob_df.round(2).to_dict(orient='index')

for key in probs.keys():
    probs[key] = list(probs[key].values())

probs['exit'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [87]:
# declare possible states in a list 

ALL_STATES = ['checkout', 'dairy', 'drinks', 'exit', 'fruit', 'spices']

In [ ]:
#CUSTOMER loop test

state = 'entrance'

while state != 'exit':
    state = random.choices(population=ALL_STATES, weights=probs[state])[0]
    print(state)

# 4 Creating Classes

Python class is more or less a function that creates objects

attributes can be modified via calling with dot function 

What will the customer do:
    be assigned a name (f.fakey), starting state (entrance), status (active/inactive)
    when customer is active choose a next state
    when state is 'exit' customer status is 'inactive'

    self referral active - customer 'NAME' is currently shopping and at checkout/in XXX aisle'
    self referral inactive - customer 'NAME' was at store for XX minutes and left at XX time


What will the supermarket do:
    Open at XX:XX time

    count through clock at minute intervals

    Allow 0-5 new customers to enter at a time
    assign each new customer a unique id
    count active customers
    max 10 active customers at a time, when 10 active, no new customers to enter store

    Each minute store is open transition customers to a new state

    At each minute record location of each active customer as line/time/aisle in csv

    no new customers -5min to close
    move customers to checkout -2min closing

    Close at XX:XX time

simulation for if __name__ == "__main__":
    for loop with (i in range opening time) 
        add 1 minute to counter
        add new active customers, 
        transition current active customers to new state
        mark active in exit and inactive



In [52]:
class Customer:
    """
    a single customer that moves through the supermarket
    in MCMC simulation
    """

    def __init__(self, customer_id):
        self.name = f.name()
        self.customer_id = customer_id
        self.state = 'entrance'
        self.is_active = true

    def get_name(self):
        return self.name

    def get_id(self):
        return self.customer_id    

    def next_state(self):
        if self.isactive == True:
            self.state = random.choices(population=ALL_STATES, weights=probs[state])[0]
            self.is_active()

    def get_state(self):
        return self.state    
 
    def is_active(self) -> bool: 
        if self.state != 'exit':
            return True

    def __repr__(self) -> str:
        return f'Welcome to the store {self.name}'

In [ ]:
# simulation dummy round for customer class
customer_id = 0
customer_count = 0
n = np.random.poisson(5)
customer_list = []
for i in range(n):
    customer_id += 1
    customer_count += 1
    new_customer = Customer(customer_id)
    customer_list.append(new_customer.get_name())
    print(f'{new_customer.get_name()} with id {new_customer.get_id()} has entered the store and is in the {new_customer.get_state()}')

In [46]:
# CURRENT WORKING CLASS TO BUILD UP FROM

class Supermarket:
    """
    an MCMC simulation that moves customers throughout the store during store opening times
    
    Parameters
    -----
    market_name: str
        what is this parameter

    Returns
    ----
    open_for_business: bool

    """

    def __init__(self, market_name, opening,  closing, max_customers=15, ave_new_customers=5):
        self.market_name = market_name
        self.opening = datetime.strptime(opening, '%H:%M')
        self.closing = datetime.strptime(closing, '%H:%M')
        self.max_customers = max_customers
        self.ave_new_customers = ave_new_customers
            #---#
        self.active_customers = []
        self.customer_id = 0
        self.current_time = self.opening
        self.customer_list = []
        self.customer_daily_count = 0
        
    def open_store(self):
        """ 
        initialises the store to start counter and begin accepting customers  
        """
        self.open_for_business = True
        while self.current_time != self.closing:
            self.timer()
            self.get_customers()
            #run new_customer.next_state() for active_customers

    def open_for_business(self):
    """ limiting setting to stop new customers entering store too late and stop too many being in store at same time """
        if current_time != self.closing - timedelta(minutes=5): #or customer_limit = false
            return True

    # def customer_limit(self):
        #if len(self.active_customers) >= 15
            #return True
            
    def timer(self):
            self.current_time += timedelta(minutes=1)
        
    def get_customers(self):
        if self.open_for_business == True:
            n = np.random.poisson(self.ave_new_customers)
            for i in range(n):
                self.customer_id += 1
                self.customer_daily_count += 1
                new_customer = Customer(self.customer_id)
                self.customer_list.append(new_customer.get_name())


# new customers generated, next step is to change customer state with each 

In [47]:
Asda = Supermarket('Asda', '07:00', '07:15')

In [48]:
Asda.open_store()

In [ ]:
Asda.customer_list

In [63]:


class Supermarket:
    """
    an MCMC simulation that moves customers through the store during store opening times
    """
    
    def __init__(self, market_name, opening,  closing, max_customers=15):
        self.market_name = market_name
        self.opening = datetime.strptime(opening, '%H:%M')
        self.closing = datetime.strptime(closing, '%H:%M')
        self.max_customers = max_customers
        #self.active_customers = 0
        self.customer_id = 0
        self.customer_list = []
        self.customer_daily_count = 0
        

    def open_store(self):
        
        current_time = self.opening
        self.open_for_business = True


    def store_closes(self):
        if current_time = self.closing - timedelta(minutes=2):
            self.open_for_business = False
            if customer.state != 'checkout':
                customer.state = 'checkout'

    def get_time(self):
        return current_time

    def get_customers(self):
        ave_new_customers = 5
        n = np.random.poisson(ave_new_customers)
        while open_for_business:
            for i in range(n):
                customer_id += 1
                customer_count += 1
                new_customer = Customer(customer_id)
                customer_list.append(new_customer.get_name())
        return      

    def customer_count(self):
        return len(self.active_customers)   

    def customer_limit(self):
        if self.customer_count == 15
            return True

    def next_minute(self):
        current_time += timedelta(minutes=1)
        self.get_customers()      

    def open_for_business(self):
        if current_time != self.closing - timedelta(minutes=5) or customer_limit = false:
            return True

    def __repr__(self):
        return f'Welcome to {self.market_name}'


    

In [ ]:
simulation plan
""" 
Simulation to run through opening times of store
final output is CSV file with list of customers and location each minute of opening 
"""
if __name__ == "__main__":
    my_supermarket = Supermarket('Asda', '07:00', '17:00')
    open.my_supermarket



In [69]:
Asda = Supermarket('Asda', '07:00', '17:00', 5)

In [66]:
Asda.store_opens()

'Asda is open at 06:59'

In [ ]:
class Supermarket:
    """
    an MCMC simulation that moves customers throughout the store during store opening times
    """
    def __init__(self, opening,  closing, max_customers=15):
        self.opening = opening
        self.closing = closing
        self.max_customers = max_customers
        self.customers = []
        self.customer_index = 0

    def open_store(self):
        current_time = self.opening - timedelta(minutes=1)

    def next_minute(self):
            current_time = current_time + timedelta(minutes=1)
            run self.get_customers 
            customers.next_state 


    def closing_time(self):
        if open_for_business = False
            if customer.state != 'checkout':
                customer_state = 'checkout'

    def get_time(self):
        return current_time

    def get_customers:
        while open_for_business:
            for i in np.random.poisson(lamba)
                customer_count +1 add to list
                customer_index +1

    def customer_limit(self):
        if customer_count == 15
            return True

    def open_for_business(self):
        if current_time != closing - timedelta(minutes=5) and customer_limit = false:
            return True

    def next_minute(self):
            current_time = current_time + timedelta(minutes=1)
            run self.get_customers 
            customers.next_state 
            

    def current_customers(self):
        print customer name
        print customer location

    def get_customers(self):
            no new customers
        else 
            new_customer = Customer()
            self.customer_index += 1

    def closing_store(self):
        if time -2 to closing:
            move customers to checkout




Action_1: Supermarket opens
    Action_1.1 -> set current time as opening
    Action_1b -> set shop_status = open for business
    Action_1c -> run the get customers function
                        Action_1.1

    WHEN OPEN FOR BUSINESS

count by one minute Increments
with each count move customers one state

customer 

enters store 
moves state with each count by supermarket
leaves store when 

In [ ]:
#datetimeplay
opening = '07:00'
time = datetime.strptime(opening, '%H:%M')
#time.hour
new_time = time + timedelta(minutes=1)
#new_time

#datetime.strptime(opening, '%H:%M')
#datetime.strptime()

new_time

# X Adding Minute-By-Minute Customer Data

In [3]:
df_fri = pd.read_csv('./data/friday_exported.csv', sep=",", index_col='timestamp', parse_dates=True)

In [4]:
df_fri

,customer_no,location_1
timestamp,,
2019-09-06 07:00:00,1,dairy
2019-09-06 07:00:00,2,drinks
2019-09-06 07:00:00,3,fruit
2019-09-06 07:01:00,2,checkout
2019-09-06 07:01:00,4,drinks
...,...,...
2019-09-06 20:50:00,1421,exit
2019-09-06 07:43:00,55,exit
2019-09-06 08:26:00,148,exit


In [5]:
df_fri.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8153 entries, 2019-09-06 07:00:00 to 2019-09-06 15:51:00
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_no  8153 non-null   int64 
 1   location_1   8153 non-null   object
dtypes: int64(1), object(1)
memory usage: 191.1+ KB


In [6]:
df_fri = df_fri.groupby('customer_no').resample('1min').ffill()

In [11]:
df_fri.loc[df_fri['customer_no']==1]

customer_no location_1
customer_no timestamp                                  
1           2019-09-06 06:59:00            1   entrance
            2019-09-06 07:00:00            1      dairy
            2019-09-06 07:01:00            1      dairy
            2019-09-06 07:02:00            1      dairy
            2019-09-06 07:03:00            1      dairy
            2019-09-06 07:04:00            1     spices
            2019-09-06 07:05:00            1   checkout
            2019-09-06 07:06:00            1       exit